In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.249698,True,24,1,41.0
1,4.312242,True,19,1,38.0
2,6.750554,False,35,1,42.0
3,8.375361,True,30,1,40.0
4,8.313632,True,20,1,44.0


In [3]:
df.describe()


,weight_pounds,mother_age,plurality,gestation_weeks
count,9992.000000,10000.000000,10000.000000,9890.000000
mean,7.296458,27.353100,1.034300,38.728817
std,1.316851,6.226194,0.191643,2.540016
min,0.705479,13.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.588395,50.000000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5078
False    4922
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [10]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6670 samples, validate on 742 samples
Epoch 1/10
6670/6670 [==============================] - 2s 339us/sample - loss: 4.9792 - mae: 1.2665 - mse: 4.9792 - val_loss: 1.5167 - val_mae: 0.9596 - val_mse: 1.5167
Epoch 2/10
6670/6670 [==============================] - 1s 150us/sample - loss: 1.5636 - mae: 0.9787 - mse: 1.5636 - val_loss: 1.9388 - val_mae: 1.1292 - val_mse: 1.9388
Epoch 3/10
6670/6670 [==============================] - 1s 125us/sample - loss: 1.5119 - mae: 0.9631 - mse: 1.5119 - val_loss: 1.5543 - val_mae: 0.9680 - val_mse: 1.5543
Epoch 4/10
6670/6670 [==============================] - 1s 121us/sample - loss: 1.4772 - mae: 0.9477 - mse: 1.4772 - val_loss: 1.3804 - val_mae: 0.9104 - val_mse: 1.3804
Epoch 5/10
6670/6670 [==============================] - 1s 158us/sample - loss: 1.4419 - mae: 0.9383 - mse: 1.4419 - val_loss: 2.0860 - val_mae: 1.1788 - val_mse: 2.0860
Epoch 6/10
6670/6670 [==============================] - 2s 253us/sample - loss: 1.3923 - mae: 0.9226 - 

In [11]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  8.320786
Actual val:  8.811876612139999

Predicted val:  7.9174414
Actual val:  7.1429772888

Predicted val:  7.7996006
Actual val:  8.24969784404

Predicted val:  8.002199
Actual val:  7.87491199864

Predicted val:  8.231019
Actual val:  7.62578964258

Predicted val:  8.29964
Actual val:  7.43839671988

Predicted val:  8.236583
Actual val:  8.6200744442

Predicted val:  8.1847925
Actual val:  6.8122838958

Predicted val:  8.003249
Actual val:  6.9996768185

Predicted val:  8.378461
Actual val:  6.77260068864

